In [1]:
%%capture
!pip install dateparser
!pip install googletrans==4.0.0rc1
!pip install pandarallel
!pip install translate

from tqdm import tqdm
from tqdm.notebook import tqdm
import dateparser
import pandas as pd
import re
from datetime import datetime
from googletrans import Translator
from translate import Translator as Translator2
import numpy as np

tqdm.pandas()

In [2]:
data=pd.read_csv("https://drive.google.com/uc?id=1r3ZwNtY1f7lWrYKgC-hk0GJCcoxSuTej",sep=";")
data.head()

,date,note,auteur,avis,assureur,produit
0,06 septem...,5,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto
1,03 mai 20...,4,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto
2,21 mars 2...,5,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto
3,10 juin 2...,4,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto
4,29 janvie...,1,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto


In [3]:
#test=pd.read_csv("https://drive.google.com/uc?id=1qsapCTYbmWFQdBWOCsXdwVlvU1-gOYHn",sep=";")
#test.head()

In [4]:
#test_en=pd.read_excel("https://docs.google.com/uc?id=1xb0rs0Thrdsb9H_vnQQaxQGSoMOcx0qe")
#test_en.head()

This dataset contains reviews published by insurance customers. There are different
columns
- date: it contains the date of the publication of the review and the period of the review
experience, and it has to be cleaned.
- note: it is the number of stars given by the customer (it should be predicted in the test
dataset)
- auteur: the id of the customer
- avis: the review
- assureur: the name of the insurance
- produit: the type of insurance


In [5]:
data.describe()

,note
count,24105.000000
mean,2.847583
std,1.531368
min,1.000000
25%,1.000000
50%,3.000000
75%,4.000000
max,5.000000


## Traduction *( done with excel due to the huge time required on python )*


In [6]:
data[data["avis"].isna()]

,date,note,auteur,avis,assureur,produit
802,23 novemb...,2,durocherdesducs-49489,NaN,GMF,auto


In [7]:
data=data.replace(np.nan, " ")

In [8]:
# translator = Translator()
# def traduction(x):
#   try:
#     return translator.translate(x,src="fr",dest='en').text
#   except:
#     return None

# data["avis_en"] = data["avis"].progress_apply(lambda x: traduction(x))

In [9]:
data_en=pd.read_excel("https://docs.google.com/uc?id=1xb0rs0Thrdsb9H_vnQQaxQGSoMOcx0qe")
data_en.head()

,date,auteur,avis,assureur,produit,avis_en
0,16 novembre 2021 suite à une expérience en nov...,lognos-d-139728,Satisfait des tarif et rapide j'attends de voi...,L'olivier Assurance,auto,Satisfied with the price and fast I expect to ...
1,16 novembre 2021 suite à une expérience en nov...,paugam-l-139727,"Bon service, service réactif. Tout en ligne pr...",L'olivier Assurance,auto,"Good service, reactive service. All in practic..."
2,16 novembre 2021 suite à une expérience en nov...,rozgonyi-o-139704,L’assurance Olivier est une très bonne assuran...,L'olivier Assurance,auto,Olivier insurance is a very good insurance I a...
3,16 novembre 2021 suite à une expérience en nov...,saad-hellal-h-139697,Je suis satisfait de l'olivier assurances pas ...,L'olivier Assurance,auto,I am satisfied with the olive assurances not t...
4,15 novembre 2021 suite à une expérience en nov...,damis-s-139693,le prix est attractif par rapport à ma situati...,L'olivier Assurance,auto,The price is attractive compared to my situati...


In [10]:
data=data_en.copy(deep=True)
print(len(data))
data=data.replace(np.nan, " ")
print(len(data))

10332
10332


## Date

In [11]:
pattern="((\w+) (\w+) (\w+)) suite à une expérience en ((\w+) (\w+))"
result = re.findall(pattern, data["date"].tolist()[0])
print("comment date:",result[0][0])
print("xp date:",result[0][4])

comment date: 16 novembre 2021
xp date: novembre 2021


In [12]:
data["date_com"] = data["date"].progress_apply(lambda x: dateparser.parse(re.findall(pattern,x)[0][0]).date())
data["date_xp"] = data["date"].progress_apply(lambda x: dateparser.parse(re.findall(pattern,x)[0][4]).date())
data=data.drop(columns=['date'])

  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]

In [13]:
data.head()

,auteur,avis,assureur,produit,avis_en,date_com,date_xp
0,lognos-d-139728,Satisfait des tarif et rapide j'attends de voi...,L'olivier Assurance,auto,Satisfied with the price and fast I expect to ...,2021-11-16,2021-11-21
1,paugam-l-139727,"Bon service, service réactif. Tout en ligne pr...",L'olivier Assurance,auto,"Good service, reactive service. All in practic...",2021-11-16,2021-11-21
2,rozgonyi-o-139704,L’assurance Olivier est une très bonne assuran...,L'olivier Assurance,auto,Olivier insurance is a very good insurance I a...,2021-11-16,2021-11-21
3,saad-hellal-h-139697,Je suis satisfait de l'olivier assurances pas ...,L'olivier Assurance,auto,I am satisfied with the olive assurances not t...,2021-11-16,2021-11-21
4,damis-s-139693,le prix est attractif par rapport à ma situati...,L'olivier Assurance,auto,The price is attractive compared to my situati...,2021-11-15,2021-11-21


## Tokenisation & Correction d'Orthographe

In [14]:
%%capture
!pip install pyspellchecker
from spellchecker import SpellChecker
from nltk.tokenize import wordpunct_tokenize
import nltk
nltk.download('punkt')

In [15]:
spell = SpellChecker(language="fr")
data["avis_token"]=data["avis"].progress_apply(lambda x:list(filter(("").__ne__, [re.sub(r'[\W_]+', '', word) for word in wordpunct_tokenize(x)])))
data["avis_token_correct"]=data["avis_token"].progress_apply(lambda x:[spell.correction(i) for i in x])

  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]

In [16]:
data['avis_token_correct'] = data.progress_apply(lambda x: [j if j is not None else x["avis_token"][i] for i,j in enumerate(x["avis_token_correct"])], axis=1)

  0%|          | 0/10332 [00:00<?, ?it/s]

In [17]:
data.head()

,auteur,avis,assureur,produit,avis_en,date_com,date_xp,avis_token,avis_token_correct
0,lognos-d-139728,Satisfait des tarif et rapide j'attends de voi...,L'olivier Assurance,auto,Satisfied with the price and fast I expect to ...,2021-11-16,2021-11-21,"[Satisfait, des, tarif, et, rapide, j, attends...","[Satisfait, des, tarif, et, rapide, j, attends..."
1,paugam-l-139727,"Bon service, service réactif. Tout en ligne pr...",L'olivier Assurance,auto,"Good service, reactive service. All in practic...",2021-11-16,2021-11-21,"[Bon, service, service, réactif, Tout, en, lig...","[Bon, service, service, réactif, Tout, en, lig..."
2,rozgonyi-o-139704,L’assurance Olivier est une très bonne assuran...,L'olivier Assurance,auto,Olivier insurance is a very good insurance I a...,2021-11-16,2021-11-21,"[L, assurance, Olivier, est, une, très, bonne,...","[L, assurance, Olivier, est, une, très, bonne,..."
3,saad-hellal-h-139697,Je suis satisfait de l'olivier assurances pas ...,L'olivier Assurance,auto,I am satisfied with the olive assurances not t...,2021-11-16,2021-11-21,"[Je, suis, satisfait, de, l, olivier, assuranc...","[Je, suis, satisfait, de, l, olivier, assuranc..."
4,damis-s-139693,le prix est attractif par rapport à ma situati...,L'olivier Assurance,auto,The price is attractive compared to my situati...,2021-11-15,2021-11-21,"[le, prix, est, attractif, par, rapport, à, ma...","[le, prix, est, attractif, par, rapport, à, ma..."


In [18]:
spell = SpellChecker(language="en")
data["avis_token_en"]=data["avis_en"].progress_apply(lambda x:list(filter(("").__ne__, [re.sub(r'[\W_]+', '', word) for word in wordpunct_tokenize(x)])))
#data["avis_token_correct_en"]=data["avis_token_en"].progress_apply(lambda x:[spell.correction(i) for i in x])
data["avis_token_correct_en"]=data["avis_token_en"]

  0%|          | 0/10332 [00:00<?, ?it/s]

## Suppression Stop Words

In [19]:
%%capture
nltk.download('stopwords')
from nltk.corpus import stopwords

In [20]:
print(stopwords.words('french'))
print(stopwords.words('english'))

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aur

In [21]:
data["avis_token_correct_sw"] = [[t for t in i if not t in stopwords.words("french")] for i in tqdm(data["avis_token_correct"])]
data["avis_token_correct_en_sw"] = [[t for t in i if not t in stopwords.words("english")] for i in tqdm(data["avis_token_correct_en"])]

  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]

## Minuscules

In [22]:
data['avis']=data['avis'].str.lower()
data['avis_en']=data['avis_en'].str.lower()
data["avis_token"]=data["avis_token"].apply(lambda s: [x.lower() for x in s])
data["avis_token_correct"]=data["avis_token_correct"].apply(lambda s:[x.lower() for x in s] )
data["avis_token_en"]=data["avis_token_en"].apply(lambda s: [x.lower() for x in s])
data["avis_token_correct_en"]=data["avis_token_correct_en"].apply(lambda s: [x.lower() for x in s])
data["avis_token_correct_sw"]=data["avis_token_correct_sw"].apply(lambda s: [x.lower() for x in s])
data["avis_token_correct_en_sw"]=data["avis_token_correct_en_sw"].apply(lambda s: [x.lower() for x in s])

## Réorganisation

In [23]:
data["avis_clean_final"]=data["avis_token_correct_sw"].apply(lambda x: ' '.join(x))
data["avis_clean_final_en"]=data["avis_token_correct_en_sw"].apply(lambda x: ' '.join(x))

In [25]:
data=data[['produit','assureur','date_com','date_xp','avis','avis_token','avis_token_correct','avis_token_correct_sw','avis_en','avis_token_en','avis_token_correct_en','avis_token_correct_en_sw']]

In [26]:
data.to_excel("avisasurance_test_clean.xlsx")